In [95]:
import pandas as pd
import numpy as np

In [96]:
df1=pd.read_csv('tmdb_5000_credits.csv')
df2=pd.read_csv('tmdb_5000_movies.csv')

In [97]:
tmdb_5000_movies.csv

NameError: name 'tmdb_5000_movies' is not defined

In [ ]:
df2.head(1)

In [ ]:
# **First we will do Demographic Filering**

In [ ]:
df1.columns=['id','title ','cast','crew']
df2=df2.merge(df1,on='id')

In [ ]:
df2.head(1)

In [ ]:
C=df2['vote_average'].mean()

In [ ]:
C

In [ ]:
df2['vote_average']

In [ ]:
m=df2['vote_count'].quantile(0.9)

In [ ]:
m

In [ ]:
q_movies=df2.copy().loc[df2['vote_count']>=m]

In [ ]:
q_movies.shape

In [98]:
df2.shape

(4803, 20)

In [99]:
q_movies.shape

(481, 24)

In [100]:
def weighted_rating(x,m=m,C=C):
    v=x['vote_count']
    R=x['vote_average']
    return (v/(v+m)*R)+(m/(m+v)*C)

In [101]:
q_movies['score']=q_movies.apply(weighted_rating,axis=1)

In [102]:
q_movies['score']

1881    8.059258
662     7.939256
65      7.920020
3232    7.904645
96      7.863239
          ...   
41      5.521697
337     5.507643
193     5.459420
91      5.406234
242     5.155730
Name: score, Length: 481, dtype: float64

In [103]:
q_movies=q_movies.sort_values('score',ascending=False)

In [104]:
q_movies[['id','title','vote_count','vote_average','score']].head(10)

,id,title,vote_count,vote_average,score
1881,278,The Shawshank Redemption,8205,8.5,8.059258
662,550,Fight Club,9413,8.3,7.939256
65,155,The Dark Knight,12002,8.2,7.920020
3232,680,Pulp Fiction,8428,8.3,7.904645
96,27205,Inception,13752,8.1,7.863239
3337,238,The Godfather,5893,8.4,7.851236
95,157336,Interstellar,10867,8.1,7.809479
809,13,Forrest Gump,7927,8.2,7.803188
329,122,The Lord of the Rings: The Return of the King,8064,8.1,7.727243
1990,1891,The Empire Strikes Back,5879,8.2,7.697884


In [105]:
#these are the top 10 movies which demographic filtering has filtered out for us.

In [106]:
#** Now we will try Content based filtering

In [107]:
df2['overview'].head(1)

0    In the 22nd century, a paraplegic Marine is di...
Name: overview, dtype: object

In [108]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(stop_words='english')
df2['overview']=df2['overview'].fillna('')
tfidf_matrix=tfidf.fit_transform(df2['overview'])

In [109]:
tfidf_matrix

<4803x20978 sparse matrix of type '<class 'numpy.float64'>'
	with 125840 stored elements in Compressed Sparse Row format>

In [110]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [111]:
indices = pd.Series(df2.index, index=df2['title']).drop_duplicates()
print(indices)

title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64


In [112]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df2['title'].iloc[movie_indices]

In [113]:
get_recommendations('The Dark Knight Rises')

65                              The Dark Knight
299                              Batman Forever
428                              Batman Returns
1359                                     Batman
3854    Batman: The Dark Knight Returns, Part 2
119                               Batman Begins
2507                                  Slow Burn
9            Batman v Superman: Dawn of Justice
1181                                        JFK
210                              Batman & Robin
Name: title, dtype: object

In [114]:
get_recommendations('The Avengers')

7               Avengers: Age of Ultron
3144                            Plastic
1715                            Timecop
4124                 This Thing of Ours
3311              Thank You for Smoking
3033                      The Corruptor
588     Wall Street: Money Never Sleeps
2136         Team America: World Police
1468                       The Fountain
1286                        Snowpiercer
Name: title, dtype: object

In [115]:
from ast import literal_eval
features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(literal_eval)

KeyError: 'cast'

In [ ]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [ ]:
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names
    return []

In [ ]:
df2['director'] = df2['crew'].apply(get_director)
features = ['cast', 'keywords', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(get_list)

In [ ]:
df2[['title', 'cast', 'director', 'keywords', 'genres']].head(1)

In [ ]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [ ]:
features = ['cast', 'keywords', 'director', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(clean_data)

In [ ]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
df2['soup'] = df2.apply(create_soup, axis=1)
print(df2['soup'])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df2['soup'])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
df2 = df2.reset_index()
indices = pd.Series(df2.index, index=df2['title'])

In [ ]:
get_recommendations('The Dark Knight Rises', cosine_sim2)

In [ ]:
get_recommendations('The Godfather', cosine_sim2)